<h1 div class='alert alert-success'><center> Stacking 
 </center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

In [1]:
import time
import warnings
import os
import gc
import random
import glob
import optuna

In [2]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt 
import seaborn              as sns
import joblib               as jb

In [3]:
from GPUtil                        import showUtilization as gpu_usage
from numba                         import cuda
from psutil                        import virtual_memory
from datetime                      import datetime

In [4]:
path = 'Data/'
df_submission = pd.read_csv(path + 'sample_submission.csv')

In [15]:
name_file_csv = glob.glob(path + '/submission/*.csv')

dic_preds_mdl_pkl = dict()
    
for p_name in name_file_csv:    
    y_model_pkl_name_col  = p_name.replace('Data//submission\\', '').replace('.csv','')  
    y_model_pkl           = pd.read_csv(p_name) 
    
    dic_preds_mdl_pkl[y_model_pkl_name_col] = y_model_pkl['target']

            

In [17]:
df_stacking =pd.DataFrame(dic_preds_mdl_pkl)
df_stacking.columns.to_list()

['0.74811_rnn_02_tunning_score_0.74468_12342_06685268427567358.pkl.z',
 '0.74812_xgb_stacking_01_score_0.74466_42_07080725777960455.pkl.z',
 '0.74935',
 '0.74940',
 '0.74940_stacking_001',
 '0.74951',
 '0.74966',
 '0.74988']

0         0.737125
1         0.753974
2         0.756775
3         0.430131
4         0.715157
            ...   
539995    0.755433
539996    0.743434
539997    0.656537
539998    0.726534
539999    0.746746
Length: 540000, dtype: float64

In [53]:
xgb = df_stacking[['0.73705_xgb' ,  '0.74049_xgb']].mean(axis=1) *.0 
svn = df_stacking[[ '0.74510_rl', ]].mean(axis=1)*.05
nn  = df_stacking[['0.74940_nn']].mean(axis=1) * .95

y_pred =pd.DataFrame({'xgb': xgb,'svn':svn,'nn':nn  }).sum(axis=1)
y_pred # 0.631371

0         0.633281
1         0.665441
2         0.661355
3         0.465857
4         0.631074
            ...   
539995    0.673551
539996    0.670049
539997    0.542110
539998    0.647143
539999    0.654693
Length: 540000, dtype: float64

In [ ]:
 (sub1.target*170 + 
  sub2.target*15 + 
  sub3.target*13 + 
  sub4.target*12 + 
  sub5.target*6 + 
  sub6.target*4) /220

In [50]:
 .05 + .05 + .15 + .75

1.0

In [53]:
y_pred = df_stacking['0.74935'] * .05 + df_stacking['0.74951'] * \
.05 + df_stacking['0.74966'] * .2 + + df_stacking['0.74988'] * .70

In [54]:
#y_pred = df_stacking['0.74935'] *.1 + df_stacking['0.74951'],'0.74966', '0.74988']].mean(axis=1)


submission = pd.DataFrame({'id': df_submission.id, 'target': y_pred})
submission.to_csv(path + 'submission/stacking_001.csv', index=False)
# score kaggle: 0.85633

In [100]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, accuracy_score

In [101]:
def postprocess_separate(submission_df, test_df=None, pure_df=None):
    """Update submission_df so that the predictions for the two sides of the hyperplane don't overlap.
    
    Parameters
    ----------
    submission_df : pandas DataFrame with columns 'id' and 'target'
    test_df : the competition's test data
    pure_df : the competition's original training data
    
    From https://www.kaggle.com/ambrosm/tpsnov21-007-postprocessing
    """
    if pure_df is None: pure_df = pd.read_csv('../input/november21/train.csv')
    if pure_df.shape != (600000, 102): raise ValueError("pure_df has the wrong shape")
    if test_df is None: test_df = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
    if test_df.shape[0] != submission_df.shape[0] or test_df.shape[1] != 101: raise ValueError("test_df has the wrong shape")

    # Find the separating hyperplane for pure_df, step 1
    # Use an SVM with almost no regularization
    model1 = make_pipeline(StandardScaler(), 
                           LinearSVC(C=1e5, tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=1))
    
    model1.fit(pure_df.drop(columns=['id', 'target']), pure_df.target)
    pure_pred = model1.predict(pure_df.drop(columns=['id', 'target']))
    print((pure_pred != pure_df.target).sum(), (pure_pred == pure_df.target).sum()) # 1 599999
    
    # Find the separating hyperplane for pure_df, step 2
    # Fit a second SVM to a subset of the points which contains the support vectors
    pure_pred = model1.decision_function(pure_df.drop(columns=['id', 'target']))
    
    subset_df = pure_df[(pure_pred > -5) & (pure_pred < 0.9)]
    model2 = make_pipeline(StandardScaler(), LinearSVC(C=1e5, tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=1))
    model2.fit(subset_df.drop(columns=['id', 'target']), subset_df.target)
    pure_pred = model2.predict(pure_df.drop(columns=['id', 'target']))
    print((pure_pred != pure_df.target).sum(), (pure_pred == pure_df.target).sum()) # 0 600000
    # model2 is perfect: it predicts the correct class for all 600000 training samples
    
    pure_test_pred = model2.predict(test_df.drop(columns=['id', 'target'], errors='ignore'))
    lmax, rmin = submission_df[pure_test_pred == 0].target.max(), submission_df[pure_test_pred == 1].target.min()
    if lmax < rmin:
        print("There is no overlap. No postprocessing needed.")
        return
    # There is overlap. Remove this overlap
    submission_df.loc[pure_test_pred == 0, 'target'] -= lmax + 1
    submission_df.loc[pure_test_pred == 1, 'target'] -= rmin - 1
    print(submission_df[pure_test_pred == 0].target.min(), submission_df[pure_test_pred == 0].target.max(),
          submission_df[pure_test_pred == 1].target.min(), submission_df[pure_test_pred == 1].target.max())

In [102]:
df3_train     = jb.load('Data/pkl/df2_nb_02_train.pkl.z')
df3_test      = jb.load('Data/pkl/df2_nb_02_test.pkl.z')

In [103]:
SEED_  = [12359]
score_ = 0 
sc     = RobustScaler()


NameError: name 'X_test' is not defined

In [ ]:
model = LinearSVC(C=1e5, tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=1)
model = make_pipeline(sc, model)

In [ ]:
X = df3_train.drop('target', axis=1)
y = df3_train['target']
X_tes = df3_test

In [104]:
X.shape, y.shape

((600000, 110), (600000,))

In [ ]:
model.fit(X, y)


In [85]:
param = {'penalty': 'l2',
 'tol': 0.001697218299300974,
 'C': 0.09607999391084766,
 'max_iter': 869,
 'random_state': 12359}